# Text data

как обработка естественного языка (англ. Natural Language Processing, NLP). NLP изучает проблемы компьютерного анализа естественных языков - т.е. языков, которые для общения используют люди (а не придуманных искусственно (например, азбука Морзе - язык, придуманный искусственно). Поговорим подробнее о том, зачем нужен NLP и где именно возникает задача обработки естественного языка.

Тексты - один из самых доступных и объёмных источников данных: легко собирать и просто хранить

Например, если у вас интернет-магазин, то для анализа доступны

* текстовые описания товаров
* пользовательские комментарии
* диалоги с продавцом-консультантом в чатике

Текстовую информацию просто хранить, поэтому проекты накапливают огромные наборы данных такого рода и очень хотят извлекать из этих объёмов полезную информацию.

Как специалист по ML в начале карьеры вы, скорее всего, встретите ряд “классических” задач - например, определение тональности (настроения) текста или классификации сообщений spam/not spam - для таких задач используются подходы, основанные на подсчёте статистик по встречающимся в тексте словам.
Однако, есть и другие, более сложные задачи.

Для решения применяются различные архитектуры нейросетей (RNN, LSTM) - это мощные инструменты, которые позволяют решать сложные задачи, например:

* извлечения именованных сущностей ([NER](https://habr.com/ru/post/414175/), Named-Entity Recognizing)
* автоматизированного перевода (например, сервис *google translate* производит перевод с помощью глубоких сетей)
* Speech Recognition - распознавание речи, трансляция из аудио в текстовый вид
* Natural Language Generation - генерация текстов, например можно генерировать подписи к картинкам

У обработки естественного языка есть ряд особенностей:

* необходимо размечать большой объём данных для обучения с учителем. Допустим, хотим отделять спам-сообщения от не спама. Вам нужно найти людей, которые прочитают все смс, которые удалось собрать и отметят те из них, которые являются спамом - текстов обычно очень много и разметка данных может оказаться дорогим удовольствием
* модель, обученную на одном языке невозможно использовать для другого языка
* важен как синтаксис, так и семантика (смысл). Например, во фразе: «Вот списки студентов, которые сдали зачет по физике» определение «которые сдали зачет по физике» относится к студентам, а в предложении: «Вот списки студентов, которые лежали в шкафу у декана»  структура фразы (тот самый синтаксис) такая же, как и предыдущей - но определение уже относится не к студентам, а к листкам бумаги. От компьютера мы хотим добиться, чтобы смыл обеих фраз был “понят” одинаково хорошо.

Кроме того, для текстов на естественном языке довольно сложно проводить предобработку данных, этот этап сильно зависит от задачи, которую вы  решаете. Так, например, для задачи анализа тональности текста знаки препинания, скорее всего, не важны. Однако, для задачи извлечения именованных сущностей (именованная сущность - это имя собственное - например название организации или географического объекта) удалять знаки препинания не рекомендуется - это может привести к потере важной информации. Например если из фразы `Мы пошли обедать в “Берёзку”` если удалить все знаки препинания (кавычки) и заглавную букву в названии заведения то станет сложнее понять, что речь идёт о кафе.

Обработка текста складывается из двух этапов

* предварительная обработка текста
* векторизация текста

#### Предварительная обработка текста

Перед тем, как обучать модель, данные следует специальным образом подготовить. Подготовка данных включает в себя несколько обязательных этапов

* удалить все нерелевантные символы (например, любые символы, не относящиеся к цифро-буквенным).
* токенизировать текст, разделив его на индивидуальные слова (токены)
* удалить нерелевантные слова — например, упоминания в Twitter или URL-ы.
* перевести все символы в нижний регистр для того, чтобы слова «hello», «Hello» и «HELLO» были схлопнуты в один токен
* исправление ошибок ("молоко" и "молако" - одно слово, но разные токены, не надо так)  
* лемматизация - перевод слова в нормальную (словарную) форму (например, «машина» вместо «машиной»). Существительные должны быть приведены к единственному числу именительного падежа, глаголы - инфинитив и т.д.
* стемминг - процедура, когда от слова переходим к его корню ("помыть" и "мытый" - корень "мыт"). То есть все "помытые" заменяем на "мыт".

Все эти приёмы нужно применять с осторожностью и внимательно следить за тем, как тот или иной приём, применённый к исходному тексту, влияет на качество решения задачи (например, выявлению спама)

Для демонстрации всех этих приёмов загрузим корпус (набор текстов) с твитами о продуктах. Для каждого твита размечена эмоциональная окраска - позитивная, нейтральная или негативная. Примечание: для  обработки текста воспользуемся библиотекой nltk, которая [доступна в anaconda](https://anaconda.org/anaconda/nltk)

Загружаем датасет с результатами модерации контента

In [ ]:
import os
import shutil

import numpy as np
from google.colab import drive


run_env = os.getenv('RUN_ENV', 'COLLAB')
if run_env == 'COLLAB':
  from google.colab import drive
  ROOT_DIR = '/content/drive'
  drive.mount(ROOT_DIR)
  print('Google drive connected')
  DRIVE_DATA_DIR = 'ml_course_data'
  root_data_dir = os.path.join(ROOT_DIR, 'MyDrive', DRIVE_DATA_DIR)
else:
  root_data_dir = os.getenv('DATA_DIR', '/srv/data')

if not os.path.exists(root_data_dir):
  raise RuntimeError('Отсутствует директория с данными')
else:
  print('Содержимое директории %s: %s' % (root_data_dir, os.listdir(root_data_dir)))

Mounted at /content/drive
Google drive connected
Содержимое директории /content/drive/MyDrive/ml_course_data: ['nyt-ingredients-snapshot-2015.csv', 'insurance (1).csv', 'non_linear.csv', 'client_segmentation.csv', 'eigen.pkl', 'clustering.pkl', 'boosting_toy_dataset.csv', 'politic_meme.jpg', 'gray_goose.jpg', 'test_dataset.pkl', 'memes', 'optimal_push_time', 'sklearn_data', 'my_little_recsys', 'corpora', 'logs', 'nltk_data', 'recsys_data', 'MNIST', 'hymenoptera_data', 'pet_projects', 'ocr_dataset_sample.csv', 'geo_points.csv.gzip', 'scored_corpus.csv', 'labeled_data_corpus.csv', 'memes_stat_dataset.zip', 'als_model.pkl', 'raw_data.zip', 'json_views.tar.gz', 'test_data.csv', 'sales_timeseries_dataset.csv.gz', 'brand_tweets_valid.csv', 'brand_tweets.csv', 'Health_and_Personal_Care.jsonl.gz', 'models', 'final_dataset.zip', 'ocr_dataset.zip', 'bidmachine_logs.zip', 'meta_Health_and_Personal_Care.jsonl.gz', 'messages.db', 'user_item_views.zip', 'content_catalog.zip', 'ground_truth_dataset.p

Датасет с текстами

In [ ]:
import pandas as pd

ocr_dataset_df = (
    pd
    .read_csv(os.path.join(root_data_dir, 'ocr_dataset.zip'), compression='zip')
    .head(10000)
)

ocr_dataset_df.head()

In [ ]:
from IPython.display import clear_output

!pip install transformers
# !pip install git+https://github.com/openai/CLIP.git
clear_output()
print('Установили transformers')

Установили transformers


In [ ]:
from transformers import pipeline

ner = pipeline('ner', grouped_entities=True)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [ ]:
sample_text = """SoftBank Vision Fund 2 is leading the round, a Series C, with iPod “father” and Nest co-founder Tony Fadell (by way of Future Shape), Blisce, French entrepreneur Xavier Niel, Mirabaud, Cassius and Evolution — all previous backers — also participating. (Previous investors in the company also include DeepMind co-founders Mustafa Suleyman and Demis Hassabis, notable given the company’s early focus on data science and recommendation algorithms.) Prior to this round Dice had raised around $45 million, according to PitchBook estimates."""

ner(sample_text)

[{'entity_group': 'ORG',
  'score': np.float32(0.9734502),
  'word': 'SoftBank Vision Fund 2',
  'start': 0,
  'end': 22},
 {'entity_group': 'ORG',
  'score': np.float32(0.96276426),
  'word': 'iP',
  'start': 62,
  'end': 64},
 {'entity_group': 'ORG',
  'score': np.float32(0.98824763),
  'word': 'Nest',
  'start': 80,
  'end': 84},
 {'entity_group': 'PER',
  'score': np.float32(0.9904026),
  'word': 'Tony Fadell',
  'start': 96,
  'end': 107},
 {'entity_group': 'ORG',
  'score': np.float32(0.9936922),
  'word': 'Future Shape',
  'start': 119,
  'end': 131},
 {'entity_group': 'ORG',
  'score': np.float32(0.86393887),
  'word': 'Blisce',
  'start': 134,
  'end': 140},
 {'entity_group': 'MISC',
  'score': np.float32(0.99727756),
  'word': 'French',
  'start': 142,
  'end': 148},
 {'entity_group': 'PER',
  'score': np.float32(0.99832183),
  'word': 'Xavier Niel',
  'start': 162,
  'end': 173},
 {'entity_group': 'ORG',
  'score': np.float32(0.9702957),
  'word': 'Mirabaud',
  'start': 175,

# NLTK

In [ ]:
nltk_data_dir = os.path.join(root_data_dir, 'nltk_data')
if not os.path.exists(nltk_data_dir):
  os.makedirs(nltk_data_dir)
  print('Директория %s создана', nltk_data_dir)
logs_dir = os.path.join(root_data_dir, 'logs')
if not os.path.exists(logs_dir):
  os.makedirs(logs_dir)
print('Подготовили директорию для nltk %s' % nltk_data_dir)

Подготовили директорию для nltk /content/drive/MyDrive/ml_course_data/nltk_data


In [ ]:
!pip install nltk==3.6.2
clear_output()

In [ ]:
import nltk

nltk.download('punkt', download_dir=nltk_data_dir)
nltk.data.path.append(nltk_data_dir) # тут почему-то корневую надо указывать ¯\_(ツ)_/¯

[nltk_data] Downloading package punkt to
[nltk_data]     /content/drive/MyDrive/ml_course_data/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import nltk

print('== Исходный текст== \n%s\n\n' % sample_text)

tokenized_str = nltk.word_tokenize(sample_text)

print('== Токенизированный текст==\n%s' % tokenized_str)

== Исходный текст== 
SoftBank Vision Fund 2 is leading the round, a Series C, with iPod “father” and Nest co-founder Tony Fadell (by way of Future Shape), Blisce, French entrepreneur Xavier Niel, Mirabaud, Cassius and Evolution — all previous backers — also participating. (Previous investors in the company also include DeepMind co-founders Mustafa Suleyman and Demis Hassabis, notable given the company’s early focus on data science and recommendation algorithms.) Prior to this round Dice had raised around $45 million, according to PitchBook estimates.


== Токенизированный текст==
['SoftBank', 'Vision', 'Fund', '2', 'is', 'leading', 'the', 'round', ',', 'a', 'Series', 'C', ',', 'with', 'iPod', '“', 'father', '”', 'and', 'Nest', 'co-founder', 'Tony', 'Fadell', '(', 'by', 'way', 'of', 'Future', 'Shape', ')', ',', 'Blisce', ',', 'French', 'entrepreneur', 'Xavier', 'Niel', ',', 'Mirabaud', ',', 'Cassius', 'and', 'Evolution', '—', 'all', 'previous', 'backers', '—', 'also', 'participating', '

Отфильтруем знаки пунктуации, токены приведём к нижнему регистру

In [ ]:
tokens = [i.lower() for i in tokenized_str if ( i not in string.punctuation )]
print(tokens)

Удаляем стоп-слова, список которых для русского языка можно получить как `stop_words = nltk.corpus.stopwords.words('russian')`. Стоп-слова это "мусорные" слова которые встречаются чрезычайно часто (в каждом предложении) поэтому не несут в себе никакой информации. Такие слова, вобщем-то, нужны только для красивой речи и поэтому можем их смело удалять из текста. Например, этот список стоп-слов я нагуглил в интернете.

In [ ]:
stop_words = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd",
    'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers',
    'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which',
    'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
    'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if',
    'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out',
    'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why',
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not',
    'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'shold',
    "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't",
    'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
    'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
    'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
]

filtered_tokens = [i for i in tokens if ( i not in stop_words )]

print(filtered_tokens)

In [ ]:
import nltk
import string
import pandas as pd

# дополнительный словарь со знаками пунктуации
nltk.download('punkt', download_dir='.')

df = pd.read_csv('data/brand_tweets.csv', sep=',', encoding='utf8')
# удаляем строки, в которых отсутствует текст твита
df.drop(df[df.tweet_text.isnull()].index, inplace=True)
print(df.shape)

df.head()

#### Домашнее задание

Реализуем пайплайн в виде функции, при помощи которой обработаем все текстовые описания. Для каждого описания
* проводим токенизацию
* удаляем пунктуацию
* приводим к нижнему регистру
* удаляем стоп-слова


Примените процедуру токенизации к файлу brand_tweets.csv

Сколько уникальных токенов получилось?

In [ ]:
def tokenize_text(raw_text: str):
    """Функция для токенизации текста

    :param raw_text: исходная текстовая строка
    """
    filtered_tokens = []
    # -- ВАШ КОД ТУТ --

    filtered_tokens = [i.lower() for i in raw_text.split() if ( i not in string.punctuation )]

    # -----------------
    return filtered_tokens

# применяем функцию в датафрейму с помощью метода .apply()
tokenized_tweets= df.tweet_text.apply(tokenize_text)

# добавляем новую колонку в исходный датафрейм
df = df.assign(
    tokenized=tokenized_tweets
)

df.tokenized.head()

Следующий шаг - привести слово в нормальную (словарную) форму. Для русского языка мы уже проводить нормализацию можно с помощью модуля **pyMorphy**, который отлично подходит для русского языка

<pre>
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
parsed_token = morph.parse(word)
normal_form = parsed_token[0].normal_form
</pre>

В силу того, что наши твиты на английском языке, то этап нормализации не слишком актуален.



### Векторизация текста: Bag of Words


Мы умеем подготавливать текст к обработке: приводить слова к начальным формам, разделять текст на токены, удалять "мусорные" токены (стоп-слова). Однако, мы знаем, что нейросети работают не с текстом, а с числами. Давайте разбираться, как переводить токены в числа, то есть с тем, как работает векторизация

Bag of Words - это способ перейти от набора токенов к численному вектору. Алгоритм векторизации текста по модели BoW:

1. определяем количество $N$ различных токенов во всех доступных текста - так называемый "словарь"
1. присваиваем каждому токену случайный номер от $0$ до $N$
1. для каждого документа $i$ формируем вектор размерности $N$ - ставим на позицию $j$ количество вхождений токена с номером $j$, которые содержатся в тексте $i$.

Каждый токен мы по сути представляем в виде вектора размерности $N$, который состоит из нулей и всего одной единицы, такое кодирование называется *One-Hot encoding*. А каждый документ это "сумма" всех one-hot векторов входящих в него токенов

Такой подход хорошо иллюстрируется картинкой:

![bow](img/bow.png)

Для каждого элемента получаем вектор из нулей и единиц. При этом размерность словаря обычно составляет несколько десятков тысяч, а количество токенов в одном документе несколько десятков - то есть нулей значительно больше, чем единиц - такие данные называются *разреженными*.

В таком виде данные уже пригодны для работы с нейросетью или любым другим алгоритмом ML, однако есть несколько довольно простых и полезных вещей, которые мы можем сделать и без нейросетей. Давайте сначала разберем их, а потом вернемся к нейросетям. Такое представление текста позволяет решать интересные задачи - например, находить самые похожие друг на друга тексты. Чтобы как-то формализовать понятие "схожести" текстов, вводится понятие *косинусного расстояния* между двумя векторами текстов $a$ и $b$ размерности $N$. С этой метрикой вы [можете познакомиться в Википедии](https://ru.wikipedia.org/wiki/Векторная_модель#Косинусное_сходство ), формула такая для двух векторов $a$ и $b$ с координатами $a_i$ и $b_i$ соответственно:
$$
\text{similarity} = \cos (\theta) = 1 - \frac{\sum_{i=1}^{N}a_ib_i}{\sqrt{\sum_{i=1}^{N}(a_i)^2}\sqrt{\sum_{i=1}^{N}(b_i)^2}}
$$

Интуитивное объяснение для простого случая: два документа полностью совпадают, тогда единички в них стоят на одних и тех же местах - расстояние между ними будет нулевым. Если два текста совершенно не пересекаются, то единички будут стоять на разных местах - расстояние в этом случае равно единице. Самостоятельно реализовывать функцию не нужно - есть готовая реализация в [scipy.spatial.distance.cosine](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.distance.cosine.html)

Векторизуем наш корпус (набор текстов) с помощью класса `CountVectorizer()` (то есть превращаем наборы токенов в наборы векторов)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# инициализируем объект, который токенизирует наш текст
# в качестве единственного аргимента передаём функцию, которую мы написали в Уроке 2
# на разбивает каждый документ на токены
vectorizer = CountVectorizer(tokenizer=tokenize_text)
# применяем наш объект-токенизатор к датафрейму с твитами
document_matrix = vectorizer.fit_transform(df.tweet_text.values)
# результат - матрица, в которой находятся числа, строк в мастрице столько, сколько документов
# а столбцов столько, сколько токенов
document_matrix

Класс `sklearn.feature_extraction.text.CountVectorizer` реализует алгоритм преобразования массива текстовых документов в разреженную матрицу такую, что

* число строк совпадает с количеством документов в исходном датафрейме
* количество столбцов совпадает с количеством различных токенов
* объект `CountVectorizer()` содержит в себе разные вспомогательные элементы - например, словарь соответствия токена и его номера

Полученные вектора можно использовать в алгоритмах второго уровня - например, в задаче классификации отзывов.

Пользуясь матрицей, найдем твит, который максимально похож на первый твит из набора

Вычисляем попарные схожести между элементами разреженной матрицы:

In [ ]:
from sklearn.metrics import pairwise_distances

tweet_distance = 1-pairwise_distances(document_matrix, metric="cosine")

tweet_distance.shape

Мы получили квадратную матрицy, которая содержит столько строк и столбцов, сколько документов в нашем  корпусе  (наборе текстов).

In [ ]:
import numpy as np
source_tweet_index = 0

# отсортируем твиты по “похожести” - чем похожее на source_tweet_index,
# тем ближе к началу списка sorted_similarity
sorted_similarity = np.argsort(-tweet_distance[source_tweet_index,:])

sorted_similarity

Мы получили квадратную матрицy, которая содержит столько строк и столбцов, сколько документов в нашем  корпусе  (наборе текстов).

In [ ]:
import numpy as np

# отсортируем твиты по “похожести” - чем похожее на source_tweet_index,
# тем ближе к началу списка sorted_similarity
sorted_similarity = np.argsort(-tweet_distance[source_tweet_index,:])

sorted_similarity

Мы получили вектор "схожестей", который содержит индексы похожих твитов, расположенных по убыванию схожести. Больше всего твит похож сам на себя, поэтому возьмём индекс второго по схожести элемента (и далее).

In [ ]:
print(df.iloc[0].tweet_text)
print('-------------')
print(df.iloc[sorted_similarity[1]].tweet_text)
print('-------------')
print(df.iloc[sorted_similarity[2]].tweet_text)
print('-------------')
print(df.iloc[sorted_similarity[3]].tweet_text)

Мы получили мощный инструмент для анализа текстов - например, мы случайно нашли дубликат твита

Кроме простого подхода, когда мы вычисляем счётчик вхождения токена, можно вычислять более сложную метрику TF-IDF (term frequency - inverse document frequency), которая вычисляется по следующей формуле для токена $t$ и документа $d$:
$$
\text{tf-idf}(t,d) = \text{tf}(t,d)\cdot\text{idf}(t)
$$

Где $\text{tf}(t,d)$ - элемент матрицы, полученной из `CountVectorizer()`, который мы умножаем на величину $\text{idf}(t)$.

Эта величина показывает количество документов в корпусе  (наборе текстов), в которых был встречен токен $t$:
$$
\text{idf}(t) = \log\frac{1+N}{1+\text{df(t)}} + 1
$$

где $\text{df}(t)$ - количество документов корпуса, в которых был встречен токен $t$. Таким образом мы понижаем веса у слов, которые встречаются почти во всех документах - такие токены являются неинформативными и мусорными, алгоритм понижает их "важность" для анализа.

Алгоритм TF-IDF лучше подходит для анализа текстов и даёт более высокое качество, но более затратен по вычислениям. Как выбрать между этими алгоритмами?

* если токенов менее 10000 используйте TF-IDF
* если токенов более 10000 то *попробуйте* использовать TF-IDF, если не получится - возвращайтесь к CountVectorizer

**Недостатки BoW подхода** Используя алгоритмы вроде Вag of Words, мы теряем порядок слов в тексте, а значит, тексты "i have no cows" и "no, i have cows" будут идентичными после векторизации, хотя и противоположными семантически. Чтобы избежать этой проблемы, можно сделать шаг назад и изменить подход к токенизации: например, использовать N-граммы (комбинации из N последовательных токенов). Обычно по корпусу  (набору текстов) формируются биграммы (последовательности из двух слов) или триграммы (последовательности из трёх слов)

Кроме того, текст можно разбивать не на слова, а на последовательности букв - при таком подходе опечатки будут автоматически учитываться.



#### Домашнее задание

Потренируйтесь в нахождении матрицы схожести для валидационного сета

загрузите brand_tweets_valid.csv
примените объект vectorizer, обученный на датасете brand_tweets.csv (просто скопируйте этот код из урока)
примените функцию pairwise_distances к полученной матрице

* Пользуясь матрицей схожести, полученной на предыдущем этапе, найдите top-5 твитов, похожих на твит валидационного сета с id=14.

У вас есть матрица схожести между объектами. Попробуйте решить задачу поиска дубликатов в тексте

1. Визуализируйте гистограмму значений в матрице схожести
1. Напишите функцию на Python, которая принимает индекс твита, пороговое значение (число от $0.0$ до $1.0$ и матрицу схожести, а затем выводит все твиты, схожесть которых больше, чем пороговое значение

In [ ]:
import pandas as pd

df_valid = pd.read_csv('data/brand_tweets_valid.csv', sep=',', encoding='utf8')
# удаляем строки, в которых отсутствует текст твита
df_valid.drop(df_valid[df_valid.tweet_text.isnull()].index, inplace=True)

# -- ВАШ КОД ТУТ --


# -----------------

## Другие подходы к векторизации текста: Word2Vec

Это более новый алгоритм, чем BoW.

Алгоритм предполагает, что мы  уже разбили документы на токены и готовы скормить наши токены нейросети, которая сделает из каждого токена плотный вектор-эмбеддинг.

При подходе BoW мы представляем каждый документ в виде разреженного вектора-строки, где размерность вектора соответствует количеству токенов в словаре.

Нетрудно заметить, что при таком подходе игнорируется контекст, в котором находится слово. Например, в двух предложениях "король издал указ" и  "правитель издал указ" слова *король* и *правитель* являются синонимами, потому что используются в одинаковом контекста. Подход *BoW* не сможет уловить отношение синонимии.

Эту проблему решает подход `Word Embedding`, при котором каждое слово представляет собой вектор большой размерности (обычно несколько сотен). В отличие от подхода BoW, при котором каждое слово представляет собой разреженный вектор, *word embedding* - это "плотный" вектор. Классическим алгоритмом, вычисляющим эмбеддинги (то есть "плотные" вектора) слов, является Word2Vec, предложена чешским аспирантом Томашем Миколовым в 2013 году. Эта модель позволяет формировать векторы, которые отражают взаимоотношения между словами: "король" относится к "королеве" так же как "женщина" к "мужчине"

![word_vectors](img/word_vectors.png)

Подход Word2Vec основан на интуитивно понятной гипотезе, которая называется гипотезой локальности — "слова, которые встречаются в одинаковых окружениях, имеют близкие значения". Эта гипотеза приводит к двум способам тренировки моделей: *Continious Bag of Words* (когда по контексту предсказываем слово) и *Skip Gram* - когда по слову пытаемся предсказать его контекст. Эмбеддинги, полученные с помощью обоих подходов оказываются идентичными - можно применять любой из них.

Пример контекста:

*Машинное обучение это* **класс** *методов искусственного интеллекта*

Мы видим, что из текста вырезается окно текста, слово в центре окна мы хотим предсказать, используя слова по краям "окна" (тот самый *контекст*).

На схеме представлены оба подхода:

![word2vec](img/word2vec.png)

На картинке представлен алгоритм тренировки *W2V* в виде простой нейросети:
![w2v_net](img/w2v_net.png)

На схеме слева-направо:

* Входной вектор $(x_1,\ldots,x_v)$ - слово из словаря, закодированное One-Hot
* $W_{V\times N}$ - матрица *word input* -  это эмбеддинги, которые мы обучаем
* Эмбеддинг слова контекста $(h_1,\ldots,h_N)$
* $W`_{N\times V}$ - матрица *word output* -  это тоже эмбеддинги но уже другие (они тоже обучаются в процессе)
* Выходной вектор $(y_1,\ldots,y_V)$ - скор для каждого слова из словаря размерности $V$

Мы видим два матричных перемножения - на самом деле W2V представляет собой очень простую нейронную сеть прямого распространения, *feed forward*.

На схеме видны две матрицы-скрытые слои. На самом деле это эмбеддинги контента, которые мы обучаем, каждая строка - эмбеддинг размерности N. Матрица эмбеддингов размером (ЧИСЛО СЛОВ В СЛОВАРЕ) X (РАЗМЕРНОСТЬ ЭМБЕДДИНГА) в начале обучения инициализируется рандомными числами, которые “превращаются” в осмысленные эмбеддинги, пока сеть обучается методом обратного распространения ошибки

На последнем слое мы получаем скоры для каждого слова из словаря. Скор (от англ score) с индексом i - это “уверенность” сети в том, что слово i может быть в контексте слова, которое мы прокидываем через сеть. То есть мы “кормим” сеть контекстом и уменьшаем лосс в случае, когда по контексту правильно удалось распознать слово внутри контекста. Слово с максимальным скором - это предсказание нашей сети. Зная "истинное" слово, которое мы предсказываем и то, что предсказала сеть, мы будем "подкручивать" веса эмбеддингов таким образом, чтобы лосс уменьшался и начинаем все лучше предсказывать слово по контексту.

Ниже показано, как работает, модификация *CBOW* - через нашу "сеть" пропускается каждое слово из контекста, мы пытаемся спрогнозировать слово "внутри" контекста:

![cbow](img/cbow.png)




В питоне существует модуль `gensim` который включает в себя библиотеки для обучения W2V.

Давайте применим алгоритм CBOW к нашему тексту:

In [ ]:
from gensim.models import Word2Vec
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

texts = df.tokenized.values

model = Word2Vec(texts, size=10, window=7, min_count=2, workers=4, iter=10, sg=0)

Мы обучили эмбеддинги слов. Давайте проверим, какой вектор обучился для слова `android`

Мы видим набор цифр - это вектор длины 10. Давайте найдём, какие слова соответствуют максимально похожим векторам


In [ ]:
print(model.wv.get_vector('android'))
model.wv.most_similar('android')


Мы видим, что модель обучила похожие вектора для слов `blackberry`, `iphone`  - это всё названия телефонов, то есть модель работает!

На основе векторизованных слов можно строить векторное описание целого предложения - такой алгоритм называется `doc2vec`.

Вывод: [модель W2V](http://www.1-4-5.net/~dmm/ml/how_does_word2vec_work.pdf) которая позволяет превращать схожие слова в "близкие" векторы, ориентируясь на контекст